<a href="https://colab.research.google.com/github/damianwgriggs/Quantum-Teleporter-Proof/blob/main/Teleporter_Proof.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install cirq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 68.7 MB/s eta 0:00:00


In [3]:
import cirq
import numpy as np

def encode_message(circuit, qubit_alice, bit1, bit2):
    """
    Alice encodes her two classical bits (bit1, bit2) onto her qubit.
    This operation transforms the initial Bell state into one of the four Bell states.

    The encoding rules are:
    - 00: Identity (I) gate - Do nothing.
    - 01: Pauli-X (X) gate. (Flips the second bit in the Bell basis)
    - 10: Pauli-Z (Z) gate. (Flips the first bit in the Bell basis)
    - 11: Z then X gates.
    """

    # bit2 controls the Pauli-X gate
    if bit2 == 1:
        circuit.append(cirq.X(qubit_alice))

    # bit1 controls the Pauli-Z gate
    if bit1 == 1:
        circuit.append(cirq.Z(qubit_alice))

    print(f"Alice's Message: {bit1}{bit2}")

    # This is the single qubit Alice physically sends to Bob
    circuit.append(cirq.Moment(cirq.GateOperation(cirq.I, [qubit_alice]), tags=['Quantum_Transmission']))


def run_superdense_coding(bit1, bit2):
    """
    Executes the Superdense Coding protocol for a given 2-bit message.
    """
    # --- 1. THE SETUP (Qubits) ---
    # Qubit 0 (Alice): The qubit Alice manipulates and sends.
    # Qubit 1 (Bob): The qubit Bob holds from the pre-shared entangled pair.
    q_alice, q_bob = cirq.LineQubit.range(2)
    circuit = cirq.Circuit()

    # --- 2. THE BRIDGE (Charlie creates the Bell Pair |Φ+⟩) ---
    # Create the Bell state |Φ+⟩ = (|00⟩ + |11⟩) / √2
    circuit.append([cirq.H(q_alice), cirq.CNOT(q_alice, q_bob)])
    circuit.append(cirq.Moment()) # Separator for visualization

    # --- 3. ALICE'S ENCODING & TRANSMISSION ---
    encode_message(circuit, q_alice, bit1, bit2)

    # --- 4. BOB'S DECODING (Reverse Bell Measurement) ---
    # Bob receives q_alice and performs the inverse Bell circuit: CNOT then H.
    circuit.append([cirq.CNOT(q_alice, q_bob), cirq.H(q_alice)])

    # --- 5. MEASUREMENT ---
    # Bob measures both qubits to retrieve the two classical bits.
    # The result of q_alice is bit1, and the result of q_bob is bit2.
    circuit.append(cirq.measure(q_alice, key='b1'))
    circuit.append(cirq.measure(q_bob, key='b2'))

    # --- EXECUTION ---
    simulator = cirq.Simulator()
    # Run the circuit once, as the protocol is deterministic (100% fidelity)
    result = simulator.run(circuit, repetitions=1)

    # --- VISUALIZATION ---
    print("\n--- The Superdense Coding Circuit ---")
    print(circuit)

    # --- VERIFICATION ---
    # Note: Cirq's measurement results are dictionaries.
    # The [0] is because we only ran 1 repetition.
    b1_out = result.measurements['b1'][0][0]
    b2_out = result.measurements['b2'][0][0]


    print("\n--- Mission Report ---")
    print(f"Classical Bits Sent (b1, b2): {bit1}{bit2}")
    print(f"Classical Bits Received: {b1_out}{b2_out}")

    if b1_out == bit1 and b2_out == bit2:
        print("✅ SUCCESS: 2 classical bits sent with 1 quantum bit.")
    else:
        print("❌ FAILURE: Message corrupted.")

    print("-" * 30)


if __name__ == "__main__":

    print("--- Project Q-3: Testing All 4 Messages (00, 01, 10, 11) ---")

    # Test Case 1: Send 00 (Identity)
    run_superdense_coding(bit1=0, bit2=0)

    # Test Case 2: Send 01 (Pauli-X)
    run_superdense_coding(bit1=0, bit2=1)

    # Test Case 3: Send 10 (Pauli-Z)
    run_superdense_coding(bit1=1, bit2=0)

    # Test Case 4: Send 11 (Pauli-Z then Pauli-X)
    run_superdense_coding(bit1=1, bit2=1)

--- Project Q-3: Testing All 4 Messages (00, 01, 10, 11) ---
Alice's Message: 00

--- The Superdense Coding Circuit ---
0: ───H───@───────I───@───H─────────M('b1')───
          │           │
1: ───────X───────────X───M('b2')─────────────

--- Mission Report ---
Classical Bits Sent (b1, b2): 00
Classical Bits Received: 00
✅ SUCCESS: 2 classical bits sent with 1 quantum bit.
------------------------------
Alice's Message: 01

--- The Superdense Coding Circuit ---
0: ───H───@───X───I───@───H─────────M('b1')───
          │           │
1: ───────X───────────X───M('b2')─────────────

--- Mission Report ---
Classical Bits Sent (b1, b2): 01
Classical Bits Received: 01
✅ SUCCESS: 2 classical bits sent with 1 quantum bit.
------------------------------
Alice's Message: 10

--- The Superdense Coding Circuit ---
0: ───H───@───Z───I───@───H─────────M('b1')───
          │           │
1: ───────X───────────X───M('b2')─────────────

--- Mission Report ---
Classical Bits Sent (b1, b2): 10
Classical Bit